# Modeling with Bayesian Networks


## Jacinto Arias @jacintoarias


*Updated 17/10/18*



# The problem

> We are part of a Data Science team from an IoT company that has developed a new Temperature sensor to monitor data center rooms in order to detect and predict early refrigeration failures. The sensors were installed a year ago, and we need to create a model that would be able to detect specific heat outbreaks or global heating problems in the room.

> We are provided with a dataset consisting on a number of measures per day during the previous year for a total of 3 sensors that have been places in strategic places across room.

### Before starting, we need some depedencies

In [ ]:
library(tidyverse)
options(repr.plot.width=6, repr.plot.height=4)

library(lubridate)

library(bnlearn)
library(networkD3)

In [ ]:
plotD3bn <- function(bn) {
  varNames <- nodes(bn)

  # Nodes should be zero indexed!
  links <- data.frame(arcs(bn)) %>%
    mutate(from = match(from, varNames)-1, to = match(to, varNames)-1, value = 1)
  
  nodes <- data.frame(name = varNames) %>%
    mutate(group = 1, size = 30)
  
  networkD3::forceNetwork(
    Links = links,  
    Nodes = nodes,
    Source = "from",
    Target = "to",
    Value = "value",
    NodeID = "name",
    Group = "group",
    fontSize = 20,
    zoom = TRUE,
    arrows = TRUE,
    bounded = TRUE,
    opacityNoHover = 1
  )
}

# The Data

You may find the sample data in `./data/heatAlarm-lvl1.csv`

In [ ]:
dataRaw <- read_csv("./data/heatAlarm-lvl1.csv")

In [ ]:
head(dataRaw)

# Exploratory analisys

We will start by studying the properties of our problem at hand. First we should look into the distribution of the dataset.

In [ ]:
summary(dataRaw)

## Conclusions

What we see:

- A complete year of measures
- Different ranges for each sensor

Additional tests:

- Data completeness
- Temporal distribution

# Number of measures by day

In [ ]:
dataRaw %>%
  group_by(Date) %>%
  summarise(Measures = n()) %>%
  ggplot(aes(x=Measures)) + geom_histogram()

## Temporal distribution

In [ ]:
dataRaw %>%
  gather("Sensor", "Measure", TS1, TS2, TS3) %>%
  ggplot(aes(x=Date, y=Measure, group=Sensor, color=Sensor)) +
  geom_line()

In [ ]:
dataRaw %>%
  gather("Sensor", "Measure", TS1, TS2, TS3) %>%
  ggplot(aes(x=Date, y=Measure, group=Sensor, color=Sensor)) +
  geom_smooth()

## Conclusions

- We observe three different distributions for each sensor

- Visually they appear to be some linear relatioship (different intercept?)

### How can we create this models as a BN?

# Latent Variables

> We can think that the sensors are measuring some real and not observable temperature from the environment. 

- This **hidden value** conditions the exact measures that the sensors capture depending on enviroment

> i.e. The sensors are located in different places in a room and the temperature varies (windows, doors, floor, ceil)

- We model this with a `Temp` latent variable that **cannot be observed** but that is captured by each sensor by following a particular dependency

- We will represent this by adding a new variable to our data with missing values

In [ ]:
dataLatent <- dataRaw %>% 
  mutate(Temp = NA) %>%
  select(Temp, TS1, TS2, TS3)

dataLatent$Temp <- as.numeric(dataLatent$Temp)

head(dataLatent)

## Build a Bayesian Network (with a latent node)

In [ ]:
heatAlarmDag1 <- model2network("[Temp][TS1|Temp][TS2|Temp][TS3|Temp]")
plotD3bn(heatAlarmDag1)

# Working with latent variables

- How can we learn the parameters?

Answer: The **EM** algorithm

1. Set an initial guess
2. **Expectation**: Compute the likelihood of the parameters generating the data, weight the data accordingly and combine weights and data.
3. **Maximization**: Improve a better estimate for the parameters using the adjusted data.
4. Repeat 2-3 until convengence

## Parametric learning with EM

1. Set an initial guess
2. **Expectation:** Impute the missing variables in the data by using Bayesian estimation from the actual model. 
3. **Maximization:**  Learning new parameters for the model by maximizing the likelihood from the imputted data (MLE).
4. Repeat 2-3 until convengence

(`bnlearn` does not have this algorithm but has a handy `inpute` function) 

The EM algorithm is then plain simple

In [ ]:
parametric.em <- function(dag, dataLatent, dataImputed, iter = 5) {
    
  fitted <- bn.fit(dag, dataImputed, method = "mle")
    
  for ( i in seq(iter)) {
   complete <- impute(fitted, data = dataLatent, method = "bayes-lw")
   fitted <- bn.fit(dag, complete, method = "mle")
  }
    
  fitted
}

## Attempt 1: Random guess (nice try)

We need an initial guess to trigger the EM algoritmh. The most straightforward is **random**.

In [ ]:
# Lets add uniform Temp 
dataImputed <- dataLatent %>% rowwise() %>% mutate(Temp = runif(1, 10, 25))
dataImputed %>%
  ggplot(aes(x=Temp)) + geom_density()

### Let's Try it!

In [ ]:
heatAlarmModel1 <- parametric.em(heatAlarmDag1, dataLatent, dataImputed, iter = 5)

In [ ]:
heatAlarmModel1

### Let's see it!

In [ ]:
# Sample some data and plot it
impute(heatAlarmModel1, data = dataLatent, method = "bayes-lw") %>%
gather("Sensor", "Measure") %>%
ggplot(aes(x=Measure, color=Sensor)) + geom_density()

### What happened?

The guess was **bad**

A bad initialization converged quick and fitted the linear gaussians to the wrong `Temp` distribution

## Attempt 2: Gaussian distribution (good luck)

- This time we will use a Gaussian distribution with the parameters computed from the sensors

In [ ]:
statistics <- dataLatent %>% 
  gather("Sensor", "Measure", TS1, TS2, TS3) %>%
  summarise( 
    mu    = mean(Measure),
    sigma = sd(Measure)
  )

dataImputed <- dataLatent %>% 
  rowwise() %>% 
  mutate(Temp = rnorm(1, statistics$mu, statistics$sigma))


In [ ]:
statistics

In [ ]:
dataImputed %>%
  ggplot(aes(x=Temp)) + geom_density()

### Let's Try it!

In [ ]:
heatAlarmModel1 <- parametric.em(heatAlarmDag1, dataLatent, dataImputed, iter = 5)

heatAlarmModel1

### Let's see it

In [ ]:
# Sample some data and plot it
impute(heatAlarmModel1, data = dataLatent, method = "bayes-lw") %>%
gather("Sensor", "Measure") %>%
ggplot(aes(x=Measure, color=Sensor)) + geom_density()

### What happened?

The guess was **bad**

We took an independent gaussian distribution using the average temperature for all the year. If you return to the temporal distribution you can see that the temperature **depends** on the **date**

In [ ]:
dataImputed$Date = dataRaw$Date

dataImputed %>%
  gather("Sensor", "Measure", Temp, TS1, TS2, TS3) %>%
  ggplot(aes(x=Date, y=Measure, group=Sensor, color=Sensor)) +
  geom_smooth()

## Attempt 3:  Sensor average (so clever)

- This time we will take the data into account and input the temperature as the average of the three sensors.

In [ ]:
dataImputed <- dataLatent %>% 
  rowwise() %>% 
  mutate(Temp = mean(c(TS1, TS2, TS3)))

head(dataImputed)

In [ ]:
dataImputed %>%
  ggplot(aes(x=Temp)) + geom_density()

### Let's Try it!

In [ ]:
heatAlarmModel1 <- parametric.em(heatAlarmDag1, dataLatent, dataImputed, iter = 10)
heatAlarmModel1

### Let's see it

In [ ]:
# Sample some data and plot it
impute(heatAlarmModel1, data = dataLatent, method = "bayes-lw") %>%
gather("Sensor", "Measure") %>%
ggplot(aes(x=Measure, color=Sensor)) + geom_density()

### What happened?

A very good **fit**

the density graph represents the sensors as scalar transformations of the latent Temp variable.

In [ ]:
dataImputed$Date = dataRaw$Date

dataImputed %>%
  gather("Sensor", "Measure", Temp, TS1, TS2, TS3) %>%
  ggplot(aes(x=Date, y=Measure, group=Sensor, color=Sensor)) +
  geom_smooth()

# Modeling the domain

- Our module captures the real temperature correctly, but it **ignores the context**

- We are not able to reproduce the current variation of temperature over time with our model, as we include the data variable in our sample

- Additional variables will help us capture this dependency

- Let's evaluate some options

## Modeling the date as the month

In [ ]:
dataByMonth <- dataRaw %>%
  mutate(Month = as.factor(month(Date, label = TRUE, abbr = TRUE)))

plot <- dataByMonth %>%
  gather("Sensor", "Measure", TS1, TS2, TS3) %>%
  ggplot(aes(x=Measure, color=Sensor, fill=Sensor)) + 
    geom_density(alpha=0.5) +
    facet_wrap(~Month)

In [ ]:
plot

## Conclusions

- **Pro** Captures differences

- **Cons** Too many dimensions

- Alternative: Find a equivalent lower dimension variable

## Modeling the date with seasons



In [ ]:
# Lubridate cant extract seasons so we need a custom function
# We better encode the states as a dict to avoid magic strings
SeasonStates <- list(
  Winter = "winter",
  Spring = "spring",
  Summer = "summer",
  Fall = "fall"
)
season <- function(date){
  WS <- as.Date("2012-12-15", format = "%Y-%m-%d") # Winter Solstice
  SE <- as.Date("2012-3-15",  format = "%Y-%m-%d") # Spring Equinox
  SS <- as.Date("2012-6-15",  format = "%Y-%m-%d") # Summer Solstice
  FE <- as.Date("2012-9-15",  format = "%Y-%m-%d") # Fall Equinox
  
  # Convert dates from any year to 2012 dates
  d <- as.Date(strftime(date, format="2012-%m-%d"))
  
  factor(
    ifelse (d >= WS | d < SE, SeasonStates$Winter,
      ifelse (d >= SE & d < SS, SeasonStates$Spring,
        ifelse (d >= SS & d < FE, SeasonStates$Summer, SeasonStates$Fall))),
    levels = SeasonStates
  )
}

dataBySeason <- dataByMonth %>%
  mutate(Season = season(Date))

plot <- dataBySeason %>%
  gather("Sensor", "Measure", TS1, TS2, TS3) %>%
  ggplot(aes(x=Measure, color=Sensor, fill=Sensor)) + 
    geom_density(alpha=0.5) +
    facet_wrap(~Season)

In [ ]:
plot

## Conclusions

- **Pro** Captures the same differences (we could test with statistics)

- **Cons** Distributions are _"less gaussian"_. Some of them seems like mixtures...

## Include in the BN

Lets include the new variables into a model and learn the parameters...

### BN with month

In [ ]:
dataLatentMonth <- dataByMonth %>%
  mutate(Temp = NA) %>%
  select(Month, Temp, TS1, TS2, TS3) %>%
  as.data.frame()

dataLatentMonth$Temp <- as.numeric(dataLatentMonth$Temp)

In [ ]:
heatAlarmDagMonth <- model2network("[Month][Temp|Month][TS1|Temp][TS2|Temp][TS3|Temp]")
plotD3bn(heatAlarmDagMonth)

### EM

In [ ]:
dataImputedMonth <- dataLatentMonth %>% 
  rowwise() %>% 
  mutate(Temp = mean(c(TS1, TS2, TS3))) %>%
  as.data.frame()

heatAlarmModelMonth <- parametric.em(heatAlarmDagMonth, dataLatentMonth, dataImputedMonth, iter = 5)
heatAlarmModelMonth

### Ploting

In [ ]:
dataImputeMonthPosterior <- impute(heatAlarmModelMonth, data = dataLatentMonth, method = "bayes-lw")

plot <- dataImputeMonthPosterior %>%
  gather("Sensor", "Measure", Temp) %>%
  ggplot(aes(x=Measure, color=Sensor, fill=Sensor)) + 
    geom_density(alpha=0.5) +
    facet_wrap(~Month)

In [ ]:
plot

### BN with season

In [ ]:
dataLatentSeason <- dataBySeason %>%
  mutate(Temp = NA) %>%
  select(Season, Temp, TS1, TS2, TS3) %>%
  as.data.frame()

dataLatentSeason$Temp <- as.numeric(dataLatentSeason$Temp)

heatAlarmDagSeason <- model2network("[Season][Temp|Season][TS1|Temp][TS2|Temp][TS3|Temp]")
plotD3bn(heatAlarmDagSeason)

### EM

In [ ]:
dataImputedSeason <- dataLatentSeason %>% 
  rowwise() %>% 
  mutate(Temp = mean(c(TS1, TS2, TS3))) %>%
  as.data.frame()

heatAlarmModelSeason <- parametric.em(heatAlarmDagSeason, dataLatentSeason, dataImputedSeason, iter = 10)
heatAlarmModelSeason

### Ploting

In [ ]:
dataImputeSeasonPosterior <- impute(heatAlarmModelSeason, data = dataLatentSeason, method = "bayes-lw")

plot <- dataImputeSeasonPosterior %>%
  gather("Sensor", "Measure", Temp, TS1, TS2, TS3) %>%
  ggplot(aes(x=Measure, color=Sensor, fill=Sensor)) + 
    geom_density(alpha=0.5) +
    facet_wrap(~Season)

In [ ]:
plot

# Modeling anomalies

- One of the most appealing aspects of Bayesian Networks is the ability to include expert knowledge

- This is almost impossible in other frameworks but it is a natural concept in this case.

- We call BNs open box models because we can understand and interpretate the parameters

- This is really useful for modelling anomalies, because we can use latent variables and expert knowledge to represent events that cannot be observed.

## How to collect data for anomalies

- If the events that cause the anomalies cannot be observed, we would not have available data to learn from

- How can we gather data for our use case to create a supervised classification problem?


<img src="./img/fire.jpg" alt="Data collection" style="width: 600px;"/>

> "Data collection.png"

## How to collect data for anomalies

- In our example we would have to destroy machines and lit the room on fire to capture data about anomalies

- That would surely have a high cost. 

- Imagine other scenarios such as rare diseases, autonomous driving or financial fraud.

- If we **know** how an anomaly looks like, we can model it directly into the model

## How to model anomalies in BNs

- We will start by modeling a global anomaly by adding a new variable Alarm

- This variable will have two states yes and no

- In this example we have been collecting data from the sensors for a year, we assume that the functioning has been correct 

> Given this, the Alarm variable will be always observed to no.

In [ ]:
# New Variables
AlarmStates <- list(
  No  = "no",
  Yes = "yes"
)

dataLatent <- dataLatentSeason %>%
  mutate(Alarm = factor(AlarmStates$No, levels = AlarmStates)) %>%
  select(Alarm, Season, Temp, TS1, TS2, TS3) %>%
  as.data.frame()

heatAlarmDag <- model2network("[Alarm][Season][Temp|Season:Alarm][TS1|Temp][TS2|Temp][TS3|Temp]")
plotD3bn(heatAlarmDag)

## EM

In [ ]:
dataImputed <- dataLatent %>% 
  rowwise() %>% 
  mutate(Temp = mean(c(TS1, TS2, TS3))) %>%
  as.data.frame()

In [ ]:
heatAlarmModel <- parametric.em(heatAlarmDag, dataLatent, dataImputed, iter = 5)

In [ ]:
heatAlarmModel$Alarm

### What happened?

Obviously the algorithm will fit all the cases to the observed population in which no alarms have occurred. 

## What can we do?

We will contact our **experts** and design a proper parametrization of the variable.

> System administrator: “We don't have many accidents, I would say that a machine brokes 1/1000 of the time, in such cases everything overheats and the room temperature is quickly raised about 10 degrees”.

## Adding expert knowledge to our model

#### Model the occurrence of a failure on the Alarm node

In [ ]:
cptAlarm <- coef(heatAlarmModel$Alarm)
print(cptAlarm)

In [ ]:
cptAlarm[] <- c(0.999, 0.001)
heatAlarmModel$Alarm <- cptAlarm
print(heatAlarmModel$Alarm)

#### Model the consecuence of a failure on the sensor nodes



In [ ]:
cgaussTemp <- coef(heatAlarmModel$Temp)
sdTemp     <- heatAlarmModel$Temp$sd
print(cgaussTemp)

In [ ]:
print(sdTemp)


In [ ]:
cgaussTemp[is.nan(cgaussTemp)] <- cgaussTemp[!is.nan(cgaussTemp)] + 10

sdTemp[is.nan(sdTemp)] <- sdTemp[!is.nan(sdTemp)]

heatAlarmModel$Temp <- list( coef = cgaussTemp, sd = sdTemp)
heatAlarmModel$Temp

## Test the anomaly

We have a completely parametrized model that we can test.

Lets do some probability queries

In [ ]:
e <- list( "Season" = SeasonStates$Winter, "TS1" = 23, "TS2" = 33, "TS3" = 29 )

query_trials <- replicate(100, cpquery(heatAlarmModel, event = Alarm == "yes", evidence = e,  method = "lw", n=10000))
query <- mean(query_trials)
print(query)

This temperature is too high for winter, so the alarm is likely to go up.

In [ ]:
e <- list( "Season" = SeasonStates$Summer, "TS1" = 23, "TS2" = 33, "TS3" = 29 )

query_trials <- replicate(100, cpquery(heatAlarmModel, event = Alarm == "yes", evidence = e,  method = "lw", n=10000))
query <- mean(query_trials)
print(query)

However this is a good fit for the higher temperatures that are registered in summer.

# Additional anomalies: Faulty sensor

We could model additional anomalies for the individual malfunctioning of each sensor. In that case we could observe either global temperature anomalies or individual malfunctions.

A common pattern is to add a new hidden discrete variable conditioning each sensor with a random distribution in the case of malfunction.

In [ ]:
# New Variables
TS1FaultStates <- list(
  No  = "no",
  Yes = "yes"
)

dataLatent <- dataLatentSeason %>%
  mutate(
    Alarm = factor(AlarmStates$No, levels = AlarmStates),
    TS1Fault = factor(TS1FaultStates$No, levels = TS1FaultStates)
  ) %>%
  select(Alarm, Season, Temp, TS1Fault, TS1, TS2, TS3) %>%
  as.data.frame()

dataImputed <- dataLatent %>% 
  rowwise() %>% 
  mutate(Temp = mean(c(TS1, TS2, TS3))) %>%
  as.data.frame()

heatAlarmDag <- model2network("[Alarm][Season][TS1Fault][Temp|Season:Alarm][TS1|Temp:TS1Fault][TS2|Temp][TS3|Temp]")

In [ ]:
plotD3bn(heatAlarmDag)

### EM

In [ ]:
heatAlarmModel <- parametric.em(heatAlarmDag, dataLatent, dataImputed, iter = 10)

In [ ]:
heatAlarmModel

## Adding expert knowledge (again)

We ask the experts about the fault tolerance of the sensors:

> IoT engineer: Our sensors are the best, they usually do not fail. The testing guys estimate just a 1/1000 failures.

### Adding expert knowledge to alarm and sensor nodes:

In [ ]:
# TODO: Please dont do this, this is not DRY...
cptAlarm[] <- c(0.999, 0.001)
heatAlarmModel$Alarm <- cptAlarm
cgaussTemp <- coef(heatAlarmModel$Temp)
sdTemp     <- heatAlarmModel$Temp$sd
cgaussTemp[is.nan(cgaussTemp)] <- cgaussTemp[!is.nan(cgaussTemp)] + 10
sdTemp[is.nan(sdTemp)] <- sdTemp[!is.nan(sdTemp)]
heatAlarmModel$Temp <- list( coef = cgaussTemp, sd = sdTemp)

### Adding expert knowledge to TS1Fault and TS1 nodes

In [ ]:
cptTS1Fault <- coef(heatAlarmModel$TS1Fault)
print(cptTS1Fault)

In [ ]:
cptTS1Fault[] <- c(0.9999, 0.0001)
heatAlarmModel$TS1Fault <- cptTS1Fault
print(heatAlarmModel$TS1Fault)

For the sensor node we will model a plain gaussian with a huge variance as a random distribution

In [ ]:
cgaussTS1 <- coef(heatAlarmModel$TS1)
sdTS1     <- heatAlarmModel$TS1$sd
print(cgaussTS1)

In [ ]:
print(sdTS1)


In [ ]:
cgaussTS1[is.nan(cgaussTS1)] <- 0
sdTS1[is.nan(sdTS1)] <- 1000000
heatAlarmModel$TS1 <- list( coef = cgaussTS1, sd = sdTS1)
heatAlarmModel$TS1

## Test the anomaly

If **only TS1** is capturing a very high temperatures the fault node will detect the anomaly and will not trigger the alarm

In [ ]:
e <- list( "Season" = SeasonStates$Winter, "TS1" = 35, "TS2" = 24, "TS3" = 20 )

query_trials <- replicate(100, cpquery(heatAlarmModel, event = Alarm == "yes", evidence = e,  method = "lw", n=10000))
query <- mean(query_trials)
print("Alarm = yes")
print(query)

e <- list( "Season" = SeasonStates$Winter, "TS1" = 35, "TS2" = 24, "TS3" = 20 )

query_trials <- replicate(100, cpquery(heatAlarmModel, event = TS1Fault == "yes", evidence = e,  method = "lw", n=10000))
query <- mean(query_trials)
print("TS1Fault = yes")
print(query)

If the three sensors are capturing unlikely temperatures, then Alarm will be triggered

In [ ]:
e <- list( "Season" = SeasonStates$Winter, "TS1" = 35, "TS2" = 37, "TS3" = 29 )

query_trials <- replicate(100, cpquery(heatAlarmModel, event = Alarm == "yes", evidence = e,  method = "lw", n=10000))
query <- mean(query_trials)
print("Alarm = yes")
print(query)

e <- list( "Season" = SeasonStates$Winter, "TS1" = 35, "TS2" = 37, "TS3" = 29 )

query_trials <- replicate(100, cpquery(heatAlarmModel, event = TS1Fault == "yes", evidence = e,  method = "lw", n=10000))
query <- mean(query_trials)
print("TS1Fault = yes")
print(query)

# Other (more complex) scenarios

Can you spot the difference in this new scenario?

In [ ]:
dataRaw <- read_csv("./data/heatAlarm-lvl2.csv")

plot <- dataRaw %>%
  gather("Sensor", "Measure", TS1, TS2, TS3) %>%
  ggplot(aes(x=Date, y=Measure, group=Sensor, color=Sensor)) +
  geom_smooth()

In [ ]:
plot

## A tip from the context

> Our experts tell us that the indoor temperature of the room is affected by the outdoor weather, specially near the windows and doors. The temperature in the interior part of the room is more stable.

Given this new knowledge we decide to place a new sensor outdoors `TSO`. Now we can have an additional component to add to our model, hopefully it will allow to mixture the indoor and outdoor temperatures.

In [ ]:
dataRaw %>%
  gather("Sensor", "Measure", TSO, TS1, TS2, TS3) %>%
  ggplot(aes(x=Date, y=Measure, group=Sensor, color=Sensor)) +
  geom_smooth()

In [ ]:
dataByMonth <- dataRaw %>%
  mutate(Month = as.factor(month(Date, label = TRUE, abbr = TRUE)))
dataByMonth %>%
  gather("Sensor", "Measure", TSO, TS1, TS2, TS3) %>%
  ggplot(aes(x=Measure, color=Sensor, fill=Sensor)) + 
    geom_density(alpha=0.5) +
    facet_wrap(~Month)

In [ ]:
dataBySeason <- dataByMonth %>%
  mutate(Season = season(Date))
dataBySeason %>%
  gather("Sensor", "Measure", TSO, TS1, TS2, TS3) %>%
  ggplot(aes(x=Measure, color=Sensor, fill=Sensor)) + 
    geom_density(alpha=0.4) +
    facet_wrap(~Season)

## Adding TSO to the model

In [ ]:
heatAlarmDag2 <- model2network("[Alarm][Season][TOut|Season][TS0|TOut][TInd|TOut:Alarm][TS1|TInd][TS2|TInd][TS3|TInd]")
plotD3bn(heatAlarmDag2)

### What will be the causation among Outdoor/Indoors and sensors?

>IoT team: Uh, yes. We instaled TS1 in a corner of the room, near the entrance. TS2 is on the ceiling and might be near a ventilation shaft, so it will probably capture a bit more of heat from the machines. TS3 should be more stable, it is located in the middle of the room.

This almost make sense visually. However, at this point we will need to test the different structures, either by learning all of them and comparing with BIC, or running a structural learning process.